In [2]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg


cfg.N_CLUSTER_RATION = 2000

cfg.EDIT_DISTANCE_THRESH_LONG = 0.25
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.25
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 5

from file_io import load_urls, dump_urls
from preprocess import make_ip_url_map
from vectorize import make_vectorize
from clustering import make_ip_cluster, make_string_distance_cluster
from regex_url import url_regex_extract, url_regex_check, regex_publish
from regex_domain import domain_regex_extract, domain_regex_check
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
import file_io

In [10]:
urls = load_urls("../../data/VirusTotalnew/malicious/total3-su.txt", csv = False)
urls = list(set(urls))
print len(urls)

2018-04-16 16:55:05 file_io.py [line:52] DEBUG	URLs Count:	1627777


294690


In [11]:
file_io.dump_urls(urls, "../../data/EXP_VirusTotal5/suspious.csv")

2018-04-16 16:55:24 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_VirusTotal5/suspious.csv


In [5]:
url_domain, url_path = preprocess.make_url_split(urls)
print len(url_domain)
print len(url_path)

129683
154858


In [6]:
url_domain = [ _ for _ in url_domain if not preprocess.check_ip(_)]
print len(url_domain)

127946


In [10]:
urls = list(url_domain)
dump_urls(urls, "../../data/EXP_VirusTotal5/urls_domain.csv")

2018-04-16 14:40:33 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/urls_domain.csv
2018-04-16 14:40:33 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_VirusTotal5/urls_domain.csv


In [11]:
cc_list = clustering.make_kmeans_cluster_mass(urls, domain=True, path=False, param=False, thresh=50000)

2018-04-16 14:40:44 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(127946, 128)
2018-04-16 14:40:44 clustering.py [line:201] DEBUG	beggin to make k-means cluster, total_data_size: 127946 n_clusters: 63
2018-04-16 14:40:44 clustering.py [line:222] DEBUG	optimal n_cluster	2
2018-04-16 14:40:48 clustering.py [line:238] DEBUG	k-means cluster done!
2018-04-16 14:40:48 clustering.py [line:266] DEBUG	Preliminary K-means clustering complete
2018-04-16 14:40:57 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(127281, 128)
2018-04-16 14:40:57 clustering.py [line:201] DEBUG	beggin to make k-means cluster, total_data_size: 127281 n_clusters: 63
2018-04-16 14:40:57 clustering.py [line:222] DEBUG	optimal n_cluster	3
2018-04-16 14:41:05 clustering.py [line:238] DEBUG	k-means cluster done!
2018-04-16 14:41:11 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(96035, 128)
2018-04-16 14:41:11 clustering.py [line:201] DEBUG	beggin to make k-means cluster

In [12]:
cc_list2 = clustering.make_hier_cluster_mass(cc_list, domain = True, path = False, param = False, dump = False)

2018-04-16 14:41:29 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(665, 128)
2018-04-16 14:41:29 clustering.py [line:149] DEBUG	beggin to make hierarchical cluster, total_data_size: 665 n_clusters: 0
2018-04-16 14:41:31 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(27305, 128)
2018-04-16 14:41:31 clustering.py [line:149] DEBUG	beggin to make hierarchical cluster, total_data_size: 27305 n_clusters: 13
2018-04-16 14:42:36 clustering.py [line:166] DEBUG	hcluster done!
2018-04-16 14:42:37 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(3941, 128)
2018-04-16 14:42:37 clustering.py [line:149] DEBUG	beggin to make hierarchical cluster, total_data_size: 3941 n_clusters: 1
2018-04-16 14:42:38 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(23465, 128)
2018-04-16 14:42:38 clustering.py [line:149] DEBUG	beggin to make hierarchical cluster, total_data_size: 23465 n_clusters: 11
2018-04-16 14:43:24 clustering.py [line:166] D

In [17]:
cc_dict = {}
for n, i in enumerate(cc_list2):
    cc_dict[n] = i
file_io._dump_kmeans_hier_cluster_data(cc_dict, "../../data/EXP_VirusTotal5/cluster_hier.json")

2018-04-16 15:03:54 file_io.py [line:154] DEBUG	cluster has been dump	../../data/EXP_VirusTotal5/cluster_hier.json


In [3]:
cc_list2 = file_io._load_kmeans_hier_cluster_data("../../data/EXP_VirusTotal5/cluster_hier.json")

2018-04-16 16:32:19 file_io.py [line:133] DEBUG	K-means/hier data has been loaded	../../data/EXP_VirusTotal5/cluster_hier.json


In [4]:
len(cc_list2)

60

In [5]:
clustering.make_string_distance_cluster_opt(data = cc_list2,
                                            n_jobs = 8,
                                            metric = "distance",  
                                            file_path = "../../data/EXP_VirusTotal5/cluster_distance.json")

batch_index 1 0/16707
batch_index 0 0/25932
batch_index 2 0/16344
batch_index 3 0/12701
batch_index 4 0/12607
batch_index 5 0/5452
batch_index 6 0/4980
batch_index 7 0/4923
batch_index 1 100/16707
batch_index 6 400/4980
batch_index 7 100/4923
batch_index 5 100/5452
batch_index 6 800/4980
batch_index 7 200/4923
batch_index 5 200/5452
batch_index 3 100/12701
batch_index 6 1200/4980
batch_index 4 100/12607
batch_index 7 300/4923
batch_index 5 300/5452
batch_index 2 100/16344
batch_index 6 2100/4980
batch_index 7 400/4923
batch_index 5 400/5452
batch_index 6 3200/4980
batch_index 7 500/4923
batch_index 3 200/12701
batch_index 1 700/16707
batch_index 6 3900/4980
batch_index 6 4000/4980
batch_index 4 200/12607
batch_index 6 4600/4980
batch_index 8 0/4400
batch_index 5 500/5452
batch_index 1 800/16707
batch_index 7 600/4923
batch_index 1 900/16707
batch_index 8 100/4400
batch_index 5 600/5452
batch_index 7 700/4923
batch_index 8 200/4400
batch_index 2 200/16344
batch_index 3 300/12701
batch_i

batch_index 13 1600/2237
batch_index 18 100/562
batch_index 1 9400/16707
batch_index 13 1700/2237
batch_index 18 200/562
batch_index 18 300/562
batch_index 18 400/562
batch_index 18 500/562
batch_index 19 0/464
batch_index 13 1900/2237
batch_index 13 2000/2237
batch_index 13 2100/2237
batch_index 20 0/455
batch_index 1 9500/16707
batch_index 20 100/455
batch_index 19 100/464
batch_index 20 200/455
batch_index 20 300/455
batch_index 20 400/455
batch_index 21 0/367
batch_index 21 100/367
batch_index 19 200/464
batch_index 21 200/367
batch_index 21 300/367
batch_index 22 0/310
batch_index 19 300/464
batch_index 19 400/464
batch_index 22 100/310
batch_index 23 0/297
batch_index 0 400/25932
batch_index 23 100/297
batch_index 22 300/310
batch_index 24 0/279
batch_index 23 200/297
batch_index 24 100/279
batch_index 25 0/276
batch_index 9 800/3941
batch_index 24 200/279
batch_index 26 0/190
batch_index 25 100/276
batch_index 25 200/276
batch_index 26 100/190
batch_index 28 0/142
batch_index 27

batch_index 4 11500/12607
batch_index 4 11600/12607
batch_index 4 11900/12607
batch_index 2 6000/16344
batch_index 4 12000/12607
batch_index 4 12100/12607
batch_index 4 12200/12607
batch_index 4 12300/12607
batch_index 4 12400/12607
batch_index 4 12500/12607
batch_index 4 12600/12607
batch_index 2 6100/16344
batch_index 0 1500/25932
batch_index 2 6200/16344
batch_index 2 6300/16344
batch_index 2 6400/16344
batch_index 2 6500/16344
batch_index 2 6600/16344
batch_index 2 6700/16344
batch_index 0 1600/25932
batch_index 2 6800/16344
batch_index 2 7000/16344
batch_index 2 7300/16344
batch_index 0 1700/25932
batch_index 2 7700/16344
batch_index 2 7800/16344
batch_index 0 1800/25932
batch_index 2 8000/16344
batch_index 2 8100/16344
batch_index 2 8200/16344
batch_index 2 8300/16344
batch_index 2 8400/16344
batch_index 2 8500/16344
batch_index 2 8700/16344
batch_index 2 8900/16344
batch_index 2 9100/16344
batch_index 2 9300/16344
batch_index 0 2000/25932
batch_index 2 9400/16344
batch_index 2 9

In [3]:
regex_list = domain_regex_extract(input_file_path = "../../data/EXP_VirusTotal5/cluster_distance.json",
                                 output_file_path = "../../data/EXP_VirusTotal5/regex_raw.txt")

2018-04-16 19:54:03 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_VirusTotal5/cluster_distance.json
2018-04-16 19:54:03 regex_domain.py [line:183] DEBUG	total cluster num:	87103
2018-04-16 19:54:03 regex_domain.py [line:185] DEBUG	big cluster:	1868
2018-04-16 19:54:03 regex_domain.py [line:187] DEBUG	small cluster:	9100
2018-04-16 19:54:03 regex_domain.py [line:188] DEBUG	single one:	76135
2018-04-16 19:54:03 regex_domain.py [line:189] DEBUG	cluster size detail:	Counter({1: 76135, 2: 6411, 3: 1833, 4: 856, 5: 522, 6: 310, 7: 226, 8: 166, 9: 106, 10: 92, 11: 71, 12: 50, 13: 36, 15: 29, 14: 28, 16: 25, 18: 21, 17: 16, 20: 13, 19: 12, 21: 9, 22: 8, 29: 6, 30: 6, 23: 4, 27: 4, 31: 4, 35: 4, 46: 4, 25: 3, 26: 3, 33: 3, 38: 3, 39: 3, 60: 3, 28: 2, 32: 2, 34: 2, 37: 2, 61: 2, 67: 2, 70: 2, 72: 2, 82: 2, 88: 2, 95: 2, 109: 2, 56: 2, 24: 1, 36: 1, 40: 1, 41: 1, 42: 1, 45: 1, 49: 1, 51: 1, 52: 1, 55: 1, 6712: 1, 1086: 1, 63: 1, 96: 1, 69: 1, 75: 1, 77: 1, 81: 1, 84: 1, 

2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^de-vorgangsnummer-\w{7,8}\.top$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'downloadstercdn', u'downloadster2cdn', u'downloadastrocdn'], 2: [u'rp', u'os']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^(:?rp|os)\.(:?downloadstercdn|downloadster2cdn|downloadastrocdn)\.com$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'clickforbest2updates4u']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^clickforbest2updates4u\.download$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'pw', u'win', u'pro'], 1: [u'click2newsafe2upgrade']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^click2newsafe2upgrade\.(:?pw|win|pro)$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: ['[A-Za-z]{3,4}'], 1: [u'nowchecksysforpcandmac']})
2018-04-16 19:54:0

2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^thegreatfreesystemos\w{0,3}content[A-Za-z]{0,12}\.(:?download|win|bid)$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'date', u'review'], 1: [u'goodsetofforcontentload']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^goodsetofforcontentload\.(:?date|review)$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'win', u'bid'], 1: [u'thebigandgreat2upgrade', u'thebigandgreat2upgrades']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^(:?thebigandgreat2upgrade|thebigandgreat2upgrades)\.(:?win|bid)$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'clickforbestandsafe247upgrades']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^clickforbestandsafe247upgrades\.download$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'bid'], 1: [u'2updateserviceformac

2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'win', u'bid'], 1: ['thebroadcenterforcontent[A-Za-z]{3,8}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^thebroadcenterforcontent[A-Za-z]{3,8}\.(:?win|bid)$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'review'], 1: ['goodnewfreecontent[A-Za-z]{4}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^goodnewfreecontent[A-Za-z]{4}\.review$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'gdn'], 1: ['read-this-message-\\w{3,5}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^read-this-message-\w{3,5}\.gdn$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'info'], 1: ['clicksafe\\w{3,9}upgrade[A-Za-z]{0,1}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^clicksafe\w{3,9}upgrade[A-Za-z]{0,1}\.info$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	default

2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^thebroadcenter[A-Za-z]{0,3}content[A-Za-z]{3,8}\.download$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'pw'], 1: ['[^\\.]{23,26}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^[^\.]{23,26}\.pw$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'000webhostapp'], 2: ['[A-Za-z]{3,8}-help-centre\\d{0,8}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^[A-Za-z]{3,8}-help-centre\d{0,8}\.000webhostapp\.com$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'click2newsafeforupgrades']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^click2newsafeforupgrades\.download$
2018-04-16 19:54:06 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'review'], 1: ['goodsetofforcontent[A-Za-z]{5,7}']})
2018-04-16 19:54:06 regex_domain.py [line:172] DEBUG	^goodsetofforco

...

2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{1,6}\.hnkangermei\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{7,8}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{7,8}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'cxvtwocr'], 2: ['c861i4[A-Za-z]{2}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^c861i4[A-Za-z]{2}\.cxvtwocr\.cn$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'cxvtwocr'], 2: ['c86\\w{4,5}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^c86\w{4,5}\.cxvtwocr\.cn$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'coinhive'], 2: ['\\w{3,5}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{3,5}\.coinhive\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdi

2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,10}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,10}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,7}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,7}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,10}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,10}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,9}']})
2018-04-16 19:54

2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,7}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['[A-Za-z]{2}dvl\\w{3,5}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^[A-Za-z]{2}dvl\w{3,5}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,7}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,7}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{6,7}\.0roptbgg23\.com$
2018-04-16 19:54:11 regex_domain.py [line:17

2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'xiazaidown'], 2: ['down\\d{6,8}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^down\d{6,8}\.xiazaidown\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'zjo1b'], 2: [u'www'], 3: [u'6fl0mt'], 4: [u'46158'], 5: ['\\w{5,6}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{5,6}\.46158\.6fl0mt\.www\.zjo1b\.download$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'xdown4'], 2: ['down050\\d{5}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^down050\d{5}\.xdown4\.com$
2018-04-16 19:54:11 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'loan'], 1: ['streaming-server1\\d{2}-location\\d{1,2}'], 2: ['\\w{7,10}']})
2018-04-16 19:54:11 regex_domain.py [line:172] DEBUG	^\w{7,10}\.streaming-server1\d{2}-location\d{1,2}\.loan$
2018-04-16 19:54:

2018-04-16 19:54:12 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'kagegame'], 2: ['\\d{3,5}'], 3: [u'com'], 4: [u'paypal']})
2018-04-16 19:54:12 regex_domain.py [line:172] DEBUG	^paypal\.com\.\d{3,5}\.kagegame\.com$
2018-04-16 19:54:12 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: ['[A-Za-z]{2,3}'], 1: [u'blogspot'], 2: [u'blogspot', u'blogluyoruz'], 3: [u'blogluyoruz']})
2018-04-16 19:54:12 regex_domain.py [line:172] DEBUG	^blogluyoruz\.(:?blogspot|blogluyoruz)\.blogspot\.[A-Za-z]{2,3}$
2018-04-16 19:54:12 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'sale'], 1: [u'xp1t'], 2: [u'uc28'], 3: [u'al2873'], 4: ['\\w{5,6}']})
2018-04-16 19:54:12 regex_domain.py [line:172] DEBUG	^\w{5,6}\.al2873\.uc28\.xp1t\.sale$
2018-04-16 19:54:12 regex_domain.py [line:171] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'sx'], 2: [u'adsl-pool'], 3: [u'118'], 4: [u'81'], 5: ['\\d{1,3}'], 6: ['\\d{2,3}']})
2018-04-16 19:54:12 regex

In [4]:
domain_regex_check(input_file_path      = "../../data/EXP_VirusTotal5/regex_raw.txt", 
            test_benign_file_path    = "../../data/sangfor/safe.csv",
            test_malicious_file_path = "../../data/EXP_VirusTotal5/suspious.csv",
            result_file_path         = "../../data/EXP_VirusTotal5/regex_result.txt", 
            n_jobs = 8)

2018-04-16 19:54:41 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_raw.txt
2018-04-16 19:54:43 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv
2018-04-16 19:54:43 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_VirusTotal5/suspious.csv


batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 0	TP(domain) 0	^mutualmysterious2110\.mutualmysterious2110\.blogspot\.[A-Za-z]{2,3}$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 7	^\d{3}\.(:?555725|555325)\.com$
batch index 0	sample index 2	FP(url) 43854	FP(domain) 19145	TP(url) 1593	TP(domain) 602	^www\.[A-Za-z]{6,7}\.com$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 36	TP(domain) 5	^[^\.]{5,6}productions\.com$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 82	TP(domain) 11	^\w{6}\.\w{4,6}\.\w{3,6}\.46utso\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 1	sample index 1	FP(url) 1	FP(domain) 1	TP(url) 314	TP(domain) 314	^[^\.]{22,25}\.(:?date|win)$
batch index 0	sample index 4	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^preparealways2systems4upgrade\.bid$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^[A-Za-z]{3}\.uglyas\.com$
batch index 2	sample index 

batch index 6	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 19	TP(domain) 19	^[A-Za-z]{5,7}\.bezoinprmotionvn\.info$
batch index 2	sample index 11	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^nowgetsoftsysformacandpc\.club$
batch index 7	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^dl\d{2}\.centerinstall\.net$
batch index 1	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^apple[^\.]{2,4}support\.com$
batch index 0	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?bestofthebestthatyouneedtohavetpday|bestofthebestthatyouneedtohavenow)\.bid$
batch index 5	sample index 5	FP(url) 0	FP(domain) 0	TP(url) 3	TP(domain) 3	^\w{4,6}\.wbf3p5\.15525\.www\.gtiuo\.win$
batch index 4	sample index 8	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebigcenter4contentsafe\.download$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^biggestupdate4new\.pro$
batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 10

batch index 1	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^tlbb\.dfztci\.15525\.www\.gtiuo\.win$
batch index 4	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 7	^error-\w{5}\.stream$
batch index 6	sample index 11	FP(url) 0	FP(domain) 0	TP(url) 126	TP(domain) 126	^\w{6,8}\.01f9y1y87\.com$
batch index 3	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 34	TP(domain) 8	^\w{8}\.firsttime-traffic\d{2}\.loan$
batch index 5	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 17	TP(domain) 17	^\d{2,3}\.(:?8888059|8888959|8888269)\.com$
batch index 0	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^alwaysdebugtoupdating\.download$
batch index 2	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 12	^\w{5,7}\.qtye1\.asia$
batch index 7	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^\w{5,6}\.60432\.15525\.www\.gtiuo\.win$
batch index 1	sample index 22	FP(url) 0	FP(domain) 0	TP(url) 368	TP(domain) 234	^edu\.cn\.[A-Za-z]{5,6}\.cn$
batch i

batch index 2	sample index 28	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^readynewsoftfree4pcandmac\.[A-Za-z]{2,4}$
batch index 0	sample index 33	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^\d{2,3}\.222593\.com$
batch index 6	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^paypal\.it\.\d{5}\.amezionuk\.org$
batch index 1	sample index 30	FP(url) 0	FP(domain) 0	TP(url) 303	TP(domain) 303	^\w{6,8}\.0roptbgg23\.com$
batch index 7	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^zpyna\.4wxy19\.15525\.www\.gtiuo\.win$
batch index 3	sample index 27	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^gov\.cn\.xytha\.cn$
batch index 4	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 89	TP(domain) 89	^\w{6}\.7ndojtv\.com$
batch index 2	sample index 29	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^press\w{1,2}up[A-Za-z]{4,5}button\.(:?win|bid)$
batch index 5	sample index 22	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^avito-user89\d{4}\.ru$
batch index 0	sample in

...

batch index 7	sample index 135	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.g78pwi\.nyljb5\.569yew\.raxheg\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 1	sample index 146	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^thegreatcentralsoft\w{1,3}content[A-Za-z]{1,8}\.download$
batch index 5	sample index 139	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^\w{2,6}\.yw566m\.15525\.www\.gtiuo\.win$
batch index 6	sample index 135	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^\d{2,3}\.(:?8888819|8888269)\.com$
batch index 4	sample index 141	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^setandperfectforyoursystemsvideo\.(:?top|pw)$
batch index 3	sample index 144	FP(url) 0	FP(domain) 0	TP(url) 892	TP(domain) 272	^\w{5,6}\.\w{3,6}\.\w{5,6}\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 0	sample index 152	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^\w{4,6}\.e5zirg\.15525\.www\.gtiuo\.win$
batch index 2	sample index 146	FP(url) 0	FP(domain) 0

batch index 7	sample index 143	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^greatglobal2upgrading\.(:?win|stream)$
batch index 1	sample index 154	FP(url) 0	FP(domain) 0	TP(url) 138	TP(domain) 138	^net-\w{2,5}\.stream$
batch index 5	sample index 147	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^blog\.\w{6}\.lchongfu\.com$
batch index 6	sample index 143	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 1	^www\.bibidown\.com$
batch index 3	sample index 152	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 4	^alindaenua\.(:?blogspot|alindaenua)\.(:?com|blogspot)\.[A-Za-z]{2,3}$
batch index 4	sample index 149	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^(:?preparingsystems4newestsofts|preparingsystems4newestsoftware|preparingsystems4newestsoft)\.(:?info|pro|top|pw)$
batch index 0	sample index 160	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^yourpushing\d{0,1}content[A-Za-z]{0,6}\.review$
batch index 2	sample index 154	FP(url) 17	FP(domain) 12	TP(url) 18	TP(domain) 12	^www\.li[A-Za-z]{4}\.cn$
b

batch index 1	sample index 162	FP(url) 0	FP(domain) 0	TP(url) 0	TP(domain) 0	^edu\.edu\.arhvd\.cn$
batch index 6	sample index 151	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^bettercentralfreeofbugs[A-Za-z]{0,2}content[A-Za-z]{4,9}\.download$
batch index 3	sample index 160	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?livecheck2updatefree4systemsmacandpc|24check4updatesfree4systemsmacandpc)\.(:?top|pro)$
batch index 4	sample index 157	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 10	^pp-[A-Za-z]{3,4}in[^\.]{5,6}\.updater-service\.com$
batch index 2	sample index 162	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^j6\d{5}\.myjino\.ru$
batch index 0	sample index 168	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^(:?firstgrade2upgrades|firstgrade2upgrade|firstgradeupgrades)\.(:?win|review)$
batch index 7	sample index 152	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?developedsoftforfreeandset|developedsoftforfreeandgo)\.club$
batch index 5	sample index 156	FP(url) 0	FP(domain) 0	TP

batch index 6	sample index 159	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^\d{1,3}\.www58123\.com$
batch index 4	sample index 165	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 5	^87374[A-Za-z]{1}\.com$
batch index 3	sample index 168	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^(:?thefreeandstablevideoforupgrade|thefreeandstablevideoforupgradenew)\.[A-Za-z]{3,8}$
batch index 2	sample index 170	FP(url) 0	FP(domain) 0	TP(url) 31	TP(domain) 12	^\w{5,6}\.\w{4,6}\.thdfr5\.nmu7o2\.ny44xh\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 7	sample index 160	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^\d{3,4}\.sicherheit-online\.verifizieren\.sicherheit-paypal-online\.(:?org|com)$
batch index 5	sample index 164	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?biggestupdate4everstableandprepared|biggestupdate4everstableandready)\.pro$
batch index 1	sample index 171	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^the[^\.]{0,1}only[^\.]{4,5}place[A-Za-z]{0,3}content[A-Za-

2018-04-16 20:05:40 file_io.py [line:249] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/regex_result.txt
2018-04-16 20:05:40 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_VirusTotal5/regex_result.txt


In [202]:
def _calc_regex_uncertainty(regex):
    uc1 = re.compile("\{\d{1,}\,(\d{1,})\}")
    uc1 = uc1.findall(regex)
    if len(uc1) !=0:
        uc1 = max([int(_) for _ in uc1])
    else:
        uc1 = 1
    uc2 = re.compile("\{(\d{1,})\}")
    uc2 = uc2.findall(regex)
    if len(uc2) !=0:
        uc2 = max([int(_) for _ in uc2])
    else:
        uc2 = 1
    uc = max(uc1, uc2)
    uc = len(regex)/float(uc)
    return uc


def _calc_pattern_len(pattern, regex):
    pattern = re.compile(pattern)
    return sum([ len(_) for _ in pattern.findall(regex)])

def _calc_regex_score(regex):
    p1 = re.compile("\,(\d{1,})\}")
    p2 = re.compile("\{(\d{1,})\}")
    uncertain = p1.findall(regex)
    uncertain += p2.findall(regex)
    uncertain = sum([int(_) for _ in uncertain])
    certain = len(regex)
    dot = re.compile("\\.")
    dot = dot.findall(regex)
    certain -= len(dot)*2
    pattern_list = ["\[A-Za-z\]\{\d{1,}\,\d{1,}\}", 
                    "\[A-Za-z\]\{\d{1,}\}",
                    "\\d\{\d{1,}\}",
                    "\\d\{\d{1,}\,\d{1,}\}",
                    "\\w\{\d{1,}\}",
                    "\\w\{\d{1,}\,\d{1,}\}",
                    "\[\^\\\.\]\{\d{1,}\,\d{1,}\}",
                    "\[\^\\\.\]\{\d{1,}\}",
                    "\(\?:",
                    "\)",
                    "\|",
                    "www",
                    "com",
                    "edu"]

    for i in pattern_list:
        certain -= _calc_pattern_len(i, regex)
        #print certain
    certain -=2
    #print uncertain, certain
    return 1 - float(uncertain)/(certain + uncertain)

_calc_regex_score("^www\.[A-Za-z]{2,3}ssa[A-Za-z]{1}\.com$")

0.4285714285714286

In [176]:
re.compile("\[A-Za-z\]\{\d{1,}\}").findall("^edu\.[A-Za-z]{5}\.cn")

['[A-Za-z]{5}']

In [209]:
df = file_io._load_check_result("../../data/EXP_VirusTotal5/regex_result.txt")
df = df.drop_duplicates("regex")
df["score"] = [_calc_regex_score(_) for _ in df.regex]
df = df.loc[df.score < 0.4]
df = df.set_index("regex")
df

2018-04-16 22:18:54 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_VirusTotal5/regex_result.txt


,fp,tp,score
regex,,,
"^www\.[A-Za-z]{6,7}\.com$",43854,1593,0.000000
"^www\d{5,6}\.com$",505,231,0.142857
"^[^\.]{17,18}\.000webhostapp\.com$",0,79,0.379310
"^www\.\w{3,4}19\d{0,5}\.com$",261,8,0.307692
"^[^\.]{25,29}\.(:?pro|pw)$",1,235,0.171429
"^www\.[A-Za-z]{7,9}\.com\.br$",41,187,0.181818
"^[^\.]{26,30}\.top$",1,166,0.032258
"^[^\.]{6,9}\.me$",366,80,0.000000
"^www\.[A-Za-z]{4,6}ar[A-Za-z]{0,1}\.com$",569,48,0.222222


In [207]:
df = file_io._load_check_result("../../data/EXP_VirusTotal5/regex_result.txt")
df = df.drop_duplicates("regex")
df["score"] = [_calc_regex_score(_) for _ in df.regex]
df = df.loc[df.score>0.5]
df = df.loc[df.fp>0]
df["ratio"] = (df.fp + 1) / (df.tp + 1)
df = df.loc[df.ratio < 0.2]
df = df.loc[df.fp <= 10]
df

2018-04-16 22:15:06 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_VirusTotal5/regex_result.txt


,fp,regex,tp,score,ratio
362,1,"^bd\d{1,2}\.52lishi\.com$",45,0.833333,0.043478
433,1,"^\w{5,9}\.000webhostapp\.com$",124,0.608696,0.016000
444,1,"^www\.9599\w{3,4}\.com$",17,0.555556,0.111111
471,1,^8888\d{3}\.com$,25,0.625000,0.076923
581,4,^c\.px9y\d{2}\.com$,38,0.750000,0.128205
765,1,^\d{3}568\.com$,19,0.571429,0.100000
814,1,^001\d{3}\.com$,41,0.571429,0.047619


In [188]:
regex_list_publish = list(df.regex)
file_io._dump_regex_list(regex_list_publish,
                 "../../data/EXP_VirusTotal5/regex_publish.txt")

2018-04-16 22:03:39 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-16 22:03:39 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal5/regex_publish.txt


In [189]:
predict_malicious, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/safe.csv",
                      regex_file_path = "../../data/EXP_VirusTotal5/regex_publish.txt")

2018-04-16 22:03:49 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-16 22:03:51 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv


batch index 0	sample index 0	hit domain 1	^bd\d{1,2}\.52lishi\.com$
batch index 1	sample index 0	hit domain 1	^\w{5,9}\.000webhostapp\.com$
batch index 2	sample index 0	hit domain 1	^www\.9599\w{3,4}\.com$
batch index 3	sample index 0	hit domain 1	^8888\d{3}\.com$
batch index 4	sample index 0	hit domain 2	^www\.[A-Za-z]{4,7}india\.com$
batch index 5	sample index 0	hit domain 3	^c\.px9y\d{2}\.com$
batch index 6	sample index 0	hit domain 1	^\d{3}568\.com$
batch index 7	sample index 0	hit domain 1	^001\d{3}\.com$
batch index 7	sample index 1	hit domain 2	^[^\.]{7,12}\.000webhostapp\.com$
batch index 7	sample index 2	hit domain 4	^www\.[A-Za-z]{2,3}ssa[A-Za-z]{1}\.com$


In [190]:
len(predict_malicious)

18

In [191]:
 predict_dict_detail

{'^001\\d{3}\\.com$': ['001233.com'],
 '^8888\\d{3}\\.com$': ['8888504.com'],
 '^[^\\.]{7,12}\\.000webhostapp\\.com$': ['polycache.000webhostapp.com',
  'hghhghghgh.000webhostapp.com'],
 '^\\d{3}568\\.com$': ['666568.com'],
 '^\\w{5,9}\\.000webhostapp\\.com$': ['polycache.000webhostapp.com'],
 '^bd\\d{1,2}\\.52lishi\\.com$': ['bd1.52lishi.com'],
 '^c\\.px9y\\d{2}\\.com$': ['c.px9y52.com', 'c.px9y94.com', 'c.px9y78.com'],
 '^www\\.9599\\w{3,4}\\.com$': ['www.9599sun.com'],
 '^www\\.[A-Za-z]{2,3}ssa[A-Za-z]{1}\\.com$': ['www.bjssae.com',
  'www.thussat.com',
  'www.ssssav.com',
  'www.wkessay.com'],
 '^www\\.[A-Za-z]{4,7}india\\.com$': ['www.seblsindia.com',
  'www.ctmaindia.com']}